In [1]:
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import datetime
import numpy
import os
import pandas
import tensorflow

# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

/Users/anonymous/anaconda3/envs/tf/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_csv = pandas.read_csv('train.csv')
train_csv['date'] = pandas.to_datetime(train_csv['date'])
train = train_csv.copy()[train_csv['date'].dt.year < 2017]

valid = train_csv.copy()[train_csv['date'].dt.year == 2017]

test_csv = pandas.read_csv('test.csv')
test = test_csv.copy()

In [3]:
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [4]:
train.tail()

,date,store,item,sales
912630,2016-12-27,10,50,60
912631,2016-12-28,10,50,43
912632,2016-12-29,10,50,68
912633,2016-12-30,10,50,63
912634,2016-12-31,10,50,64


In [5]:
valid.head()

,date,store,item,sales
1461,2017-01-01,1,1,19
1462,2017-01-02,1,1,15
1463,2017-01-03,1,1,10
1464,2017-01-04,1,1,16
1465,2017-01-05,1,1,14


In [6]:
valid.tail()

,date,store,item,sales
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62
912999,2017-12-31,10,50,82


In [7]:
test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


In [8]:
test.tail()

,id,date,store,item
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50
44999,44999,2018-03-31,10,50


In [9]:
epochs = 64
units = 30

In [10]:
def create_sequences(data, unit):
    
    data = data.loc[(data['store'] == 1) & (data['item'] == 1)]
    data = data.drop('date', axis=1).drop('store', axis=1).drop('item', axis=1)
    
    X, y = [], []
    
    for index in range(unit, len(data)):
        X.append(data.iloc[index - unit: index].to_numpy())
        y.append(data.iloc[index].to_numpy())
    
    return numpy.array(X), numpy.array(y)

In [11]:
stores, items = range(1, 2), range(1, 2)
epochs = [160, 192, 224, 256, 384, 512]
units = [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
histories = dict()

for store in stores:
    for item in items:
        for epoch in epochs:
            for unit in units:
                train_X, train_y = create_sequences(train, unit)
                valid_X, valid_y = create_sequences(valid, unit)

                model = Sequential()
                model.add(LSTM(units=unit, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
                model.add(Dropout(0.1))
                model.add(LSTM(units=unit))
                model.add(Dropout(0.1))
                model.add(Dense(1))
                model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tensorflow.keras.metrics.MeanSquaredError()])

                print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '|', (store, item, epoch, unit))
                histories[(store, item, epoch, unit)] = model.fit(train_X, train_y, epochs=epoch, validation_data=(valid_X, valid_y), verbose=0)

Metal device set to: Apple M1


2022-09-20 10:13:04.387354: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-20 10:13:04.387450: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


2022-09-20 10:13:04 | (1, 1, 160, 30)


2022-09-20 10:13:04.705707: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-20 10:13:05.817671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 10:13:05.964011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 10:13:06.045450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 10:13:06.145618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 10:13:06.288581: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 10:13:07.603822: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114

2022-09-20 10:15:35 | (1, 1, 160, 60)


...

2022-09-20 12:06:39 | (1, 1, 160, 270)


2022-09-20 12:06:40.362553: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 12:06:40.512766: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 12:06:41.015763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 12:06:41.470403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 12:06:42.612358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 12:06:51.909963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 12:06:51.967957: I tensorflow/core/grappler/optimizers/cust

2022-09-20 15:21:04 | (1, 1, 160, 300)


2022-09-20 15:21:05.795205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 15:21:05.947735: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 15:21:06.473266: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 15:21:07.019353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 15:21:08.422955: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 15:21:19.629207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-20 15:21:19.690151: I tensorflow/core/grappler/optimizers/cust

KeyboardInterrupt: 

In [12]:
for store in stores:
    for item in items:
        for epoch in epochs:
            for unit in units:
                history = histories[(store, item, epoch, unit)].history
                print((store, item, epoch, unit), history['loss'][-1], history['mean_squared_error'][-1], history['val_loss'][-1], history['val_mean_squared_error'][-1])

(1, 1, 160, 30) 26.235595703125 26.235595703125 34.001949310302734 34.001949310302734
(1, 1, 160, 60) 22.730751037597656 22.730751037597656 34.28974533081055 34.28974533081055
(1, 1, 160, 90) 24.715293884277344 24.715293884277344 32.177146911621094 32.177146911621094
(1, 1, 160, 120) 28.72620391845703 28.72620391845703 38.172828674316406 38.172828674316406
(1, 1, 160, 150) 27.116920471191406 27.116920471191406 35.89410400390625 35.89410400390625
(1, 1, 160, 180) 24.37425422668457 24.37425422668457 28.29886245727539 28.29886245727539
(1, 1, 160, 210) 22.895748138427734 22.895748138427734 24.205968856811523 24.205968856811523
(1, 1, 160, 240) 21.87909507751465 21.87909507751465 24.70415687561035 24.70415687561035
(1, 1, 160, 270) 28.43602752685547 28.43602752685547 33.12578582763672 33.12578582763672


KeyError: (1, 1, 160, 300)